## This script is to convert recorded demonstrations (from rosbag format) to samples (in txt format) to conform with the TP-GMM code.

In [25]:
import rosbag
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sys import platform

# from franka_msgs.msg import FrankaState

print(platform)
if platform == "linux":
    main_dir = "/home/erl/Disassembly Teleop/"
    TPGMM_dir = "/home/erl/Multicobot-UR10/src/tp_gmm/data/"
elif platform == "win32":
    main_dir = "C:/Users/AWasf/OneDrive/Desktop/PhD Lab Work/Disassembly-Teleop/"
    TPGMM_dir = "C:/Users/AWasf/OneDrive/Desktop/PhD Lab Work/tp_gmm/data/"


linux


### Defining the directory according to the given task_name, and thus loop over the Demons in the directory (task_name, nbDemons in dir, ref_demon)

In [26]:
# Getting the task_name from tp-gmm.py
import pickle

with open('/home/erl/Multicobot-UR10/src/tp_gmm/scripts/demons_info1.pkl', 'rb') as fp:
    demons_info = pickle.load(fp)
    print(demons_info)
task_name = demons_info['task_name']
print("task_name: ", task_name)

# 
import os

demons_names = []
demons_nums = []
for filename in os.listdir(main_dir + "Demons/" + task_name + "/action"):
    demons_names.append(filename)
    demons_nums.append(filename[filename.find('Demon'):filename.find('.bag')])

print(demons_names)
print(demons_nums)

nbdemons = len(demons_names)
print("Number of Demons: ", nbdemons)


{'task_name': 'Rbolts'}
task_name:  Rbolts
['Franka_Stage2_Rbolts_action_Demon_02_5_3.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_5.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_7.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_1.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_2.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_5.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_6.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_9.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_1.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_8.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_4.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_7.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_4.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_2.bag', 'Franka_Stage2_Rbolts_action_Demon_02_5_6.bag', 'Franka_Stage2_Rbolts_action_Demon_02_4_3.bag']
['Demon_02_5_3', 'Demon_02_4_5', 'Demon_02_4_7', 'Demon_02_5_1', 'Demon_02_5_2', 'Demon_02_5_5', 'Demon_02_4_6', 'Demon_02_5_9', 'Demon_02_4_1', 'Demon_02_5_8', 'Demon_02_4_4', 'Demon_02_5

### 3D

In [27]:
from geometry_msgs.msg import Pose, PoseArray
from copy import deepcopy

ref_demon = {'ref':'', 'ref_nbpoints': 100000, 'nbDemons': nbdemons, 'demons_nums': demons_nums} # Choosing the shortest demon to be the reference in DTW
# ref_demon = {'ref':'', 'ref_nbpoints': 0, 'nbDemons': nbdemons, 'demons_nums': demons_nums} # Choosing the longest demon to be the reference in DTW

for d in range(nbdemons):
    # task_name = "Rcover"

    # Save the demonstrations in a rosbag to visualize in RViz
    viz_demon_bag = rosbag.Bag(TPGMM_dir + "/posearrays/" + demons_names[d], 'w')
    demons_posearray = PoseArray()
    demons_posearray.header.frame_id = 'ur10_1_base_link_gmm'
    ##

    demon_bag = rosbag.Bag(main_dir + "Demons/" + task_name + "/action/" + demons_names[d], 'r')

    data_topic = "/panda_teleop/follower_state_controller/franka_states"

    dim = 1 + 3
    conc_arr_Data = np.zeros((dim,1))
    down_sample = 30 # 10
    msg_count = 0
    for topic, msg, t in demon_bag.read_messages(data_topic):
        if (msg_count % down_sample == 0):
            # Preparing the Data points matrix from the Demons
            # arr_Data = np.array([[sqrt(msg.O_T_EE_c[12]**2 + msg.O_T_EE_c[13]**2 + msg.O_T_EE_c[14]**2)], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]]) # Euc. distance of error_pose
            arr_Data = np.array([[msg.O_T_EE_c[0]], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]]) # time sample

            conc_arr_Data = np.concatenate((conc_arr_Data, arr_Data), axis=1)

            # Preparing the b matrix & A matrix for frame 1 (Position & RotationMatrix of the Starting point(frame) of the Demon w.r.t the base frame of the robot (panda_link0))
            if (msg_count == 0):
                arr_frame1_b = np.array([[0], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]])
                arr_frame1_A = np.matrix([[1,             0,             0,             0],
                                        [0, msg.O_T_EE[0], msg.O_T_EE[4], msg.O_T_EE[8]],
                                        [0, msg.O_T_EE[1], msg.O_T_EE[5], msg.O_T_EE[9]],
                                        [0, msg.O_T_EE[2], msg.O_T_EE[6], msg.O_T_EE[10]]])

            # Preparing the b matrix & A matrix for frame 2 (Position & RotationMatrix of the Ending point(frame) of the Demon w.r.t the base frame of the robot (panda_link0))
            arr_frame2_b = np.array([[0], [msg.O_T_EE[12]], [msg.O_T_EE[13]], [msg.O_T_EE[14]]])
            arr_frame2_A = np.matrix([[1,             0,             0,             0],
                                    [0, msg.O_T_EE[0], msg.O_T_EE[4], msg.O_T_EE[8]],
                                    [0, msg.O_T_EE[1], msg.O_T_EE[5], msg.O_T_EE[9]],
                                    [0, msg.O_T_EE[2], msg.O_T_EE[6], msg.O_T_EE[10]]])
            
            demon_pose = Pose()
            demon_pose.position.x = arr_Data[1]; demon_pose.position.y = arr_Data[2]; demon_pose.position.z = arr_Data[3]
            # quat = R.from_matrix(arr_frame2_A[1:,1:])
            demon_pose.orientation.w = 1.0
            
            demons_posearray.poses.append(deepcopy(demon_pose))

        msg_count += 1

    viz_demon_bag.write("/demons_posearray", demons_posearray)
    viz_demon_bag.close()        
    
    demon_bag.close()
    
    print("Number of data points in {}: {}".format(demons_nums[d], conc_arr_Data.shape[1]-1)) #, msg_count)
    # Choosing the shortest Demon to be the reference Demon
    # if (msg_count < ref_demon['ref_nbpoints']):
    #     ref_demon["ref"] = demons_nums[d]
    #     ref_demon["ref_nbpoints"] = msg_count
    if (conc_arr_Data.shape[1]-1 < ref_demon['ref_nbpoints']):
        ref_demon["ref"] = demons_nums[d]
        ref_demon["ref_nbpoints"] = conc_arr_Data.shape[1]-1 # because the DTW reduces 1 point!! # msg_count
        ref_demon["down_sample_factor"] = down_sample    
    # Choosing the longest Demon to be the reference Demon
    # if (msg_count > ref_demon['ref_nbpoints']):
    #     ref_demon["ref"] = demons_nums[d]
    #     ref_demon["ref_nbpoints"] = msg_count

    # Filling in b matrix & A matrix for frame 1
    conc_arr_frame1_b = np.ones((dim, msg_count)) * arr_frame1_b
    conc_arr_frame1_A = np.tile(arr_frame1_A, msg_count) # the .T is because the rotation is around negative y-axis

    # Filling in b matrix & A matrix for frame 2
    conc_arr_frame2_b = np.ones((dim, msg_count)) * arr_frame2_b
    conc_arr_frame2_A = np.tile(arr_frame2_A, msg_count)

    # Saving in txt files
    np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_Data.txt", conc_arr_Data[:,1:], fmt='%.5f') # cut out the 1st column (zeros) that was created only to initialize the array 
    np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame1_b.txt", conc_arr_frame1_b, fmt='%.5f')
    np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame1_A.txt", conc_arr_frame1_A, fmt='%.5f')
    np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame2_b.txt", conc_arr_frame2_b, fmt='%.5f')
    np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame2_A.txt", conc_arr_frame2_A, fmt='%.5f')

    # Add the ',' to put file in the format that TP-GMM code accepts
    with open(r"{}".format(TPGMM_dir) + demons_nums[d]+ "_sample_Data.txt", 'r') as f: # The 'r' before the directory name is to open the file as read-only
        data = f.read(); data = data.replace(' ', ',')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d]+ "_sample_Data.txt", 'w') as f:
        f.write(data)
        
    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_b.txt", 'r') as f:
        data = f.read(); data = data.replace(' ', ',')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_b.txt", 'w') as f:
        f.write(data)

    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_A.txt", 'r') as f:
        data = f.read(); data = data.replace(' ', ',')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_A.txt", 'w') as f:
        f.write(data)

    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame2_b.txt", 'r') as f:
        data = f.read(); data = data.replace(' ', ',')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame2_b.txt", 'w') as f:
        f.write(data)

    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame2_A.txt", 'r') as f:
        data = f.read(); data = data.replace(' ', ',')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d]+ "_sample_frame2_A.txt", 'w') as f:
        f.write(data)

print("Reference Demon:")
print(ref_demon)
with open('/home/erl/Multicobot-UR10/src/tp_gmm/scripts/demons_info2.pkl', 'wb') as fp:
    pickle.dump(ref_demon, fp)

Number of data points in Demon_02_5_3: 123
Number of data points in Demon_02_4_5: 158
Number of data points in Demon_02_4_7: 85
Number of data points in Demon_02_5_1: 114
Number of data points in Demon_02_5_2: 163
Number of data points in Demon_02_5_5: 122
Number of data points in Demon_02_4_6: 81
Number of data points in Demon_02_5_9: 214
Number of data points in Demon_02_4_1: 54
Number of data points in Demon_02_5_8: 57
Number of data points in Demon_02_4_4: 278
Number of data points in Demon_02_5_7: 134
Number of data points in Demon_02_5_4: 190
Number of data points in Demon_02_4_2: 219
Number of data points in Demon_02_5_6: 179
Number of data points in Demon_02_4_3: 186
Reference Demon:
{'ref': 'Demon_02_4_1', 'ref_nbpoints': 54, 'nbDemons': 16, 'demons_nums': ['Demon_02_5_3', 'Demon_02_4_5', 'Demon_02_4_7', 'Demon_02_5_1', 'Demon_02_5_2', 'Demon_02_5_5', 'Demon_02_4_6', 'Demon_02_5_9', 'Demon_02_4_1', 'Demon_02_5_8', 'Demon_02_4_4', 'Demon_02_5_7', 'Demon_02_5_4', 'Demon_02_4_2',

In [28]:
from scipy.spatial.transform import Rotation as R

print(arr_frame1_b)
print(arr_frame1_A[1:,1:])
quat = R.from_matrix(arr_frame1_A[1:,1:])

print("-----")

print(quat.as_quat())
print(arr_frame2_b)
print(arr_frame2_A[1:,1:])
quat = R.from_matrix(arr_frame2_A[1:,1:])
print(quat.as_quat())

[[0.        ]
 [0.4415556 ]
 [0.0612961 ]
 [0.48154897]]
[[ 0.96411779  0.20383847  0.17002373]
 [ 0.18378084 -0.97478082  0.12652275]
 [ 0.19152607 -0.09073573 -0.97728395]]
-----
[ 0.98946976  0.0979366   0.09134982 -0.05489292]
[[0.        ]
 [0.52047228]
 [0.33217799]
 [0.09997107]]
[[ 0.66065986  0.75055006 -0.01356102]
 [ 0.74501997 -0.65336535  0.1343133 ]
 [ 0.09194855 -0.09883864 -0.99084612]]
[ 0.90896789  0.41133981  0.02155961 -0.06412583]


### DTW the demons

In [ ]:
%matplotlib inline
# %matplotlib qt
from dtw import *

print("Reference Demon:")
ref_arr_Data = np.loadtxt(TPGMM_dir + ref_demon['ref'] + "_sample_Data.txt", delimiter=',')
print(ref_arr_Data.shape)
demons_nums_dtw = []

for d in range(nbdemons):
    
    print("DTWinng..")
    temp_arr_Data = np.loadtxt(TPGMM_dir + demons_nums[d] + "_sample_Data.txt", delimiter=',')
    print(temp_arr_Data.shape)

    DTW = dtw(temp_arr_Data[1:,:].T, ref_arr_Data[1:,:].T)
    wq = warp(DTW, index_reference=False)
    print(wq.shape)
    print(warpArea(DTW))
    print(DTW.distance)
    print(DTW.normalizedDistance)

    if (DTW.normalizedDistance >= 0.30): continue # Filter out the odd demons
    
    ## Plotting
    fig = plt.figure()
    ax = plt.axes(projection='3d')

    ax.plot(ref_arr_Data[1,:], ref_arr_Data[2,:], ref_arr_Data[3,:], label='ref')
    ax.plot(temp_arr_Data[1,:], temp_arr_Data[2,:], temp_arr_Data[3,:], label='{}'.format(demons_nums[d]))
    ax.plot(temp_arr_Data[1,wq], temp_arr_Data[2,wq], temp_arr_Data[3,wq], label='DTW {}'.format(demons_nums[d]))
    
    ax.legend()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')

    ## Saving in .txt
    temp_arr_Data = temp_arr_Data[:,wq]
    temp_arr_Data[0,:] = ref_arr_Data[0,:ref_arr_Data.shape[1]]#-1] # NOTE: It seems that with the update of 'dtw' package, this '-1' here has to be removed

    np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_Data.txt", temp_arr_Data, fmt='%.5f')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_Data.txt", 'r') as f: # The 'r' before the directory name is to open the file as read-only
        data = f.read(); data = data.replace(' ', ',')
    with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_Data.txt", 'w') as f:
        f.write(data)
    # # Re-shaping the A & b matrices .txt to be of the same shape as the reference Demon (longest or shortest)
    # temp_arr_frame1_b = np.loadtxt(TPGMM_dir + demons_nums[d] + "_sample_frame1_b.txt", delimiter=',')
    # temp_arr_frame1_A = np.loadtxt(TPGMM_dir + demons_nums[d] + "_sample_frame1_A.txt", delimiter=',')
    # temp_arr_frame2_b = np.loadtxt(TPGMM_dir + demons_nums[d] + "_sample_frame2_b.txt", delimiter=',')
    # temp_arr_frame2_A = np.loadtxt(TPGMM_dir + demons_nums[d] + "_sample_frame2_A.txt", delimiter=',')
    
    # temp_arr_frame1_b = np.ones((ref_arr_Data.shape[0], ref_arr_Data.shape[1])) * temp_arr_frame1_b[:,0].reshape(ref_arr_Data.shape[0], 1)
    # temp_arr_frame1_A = np.tile(temp_arr_frame1_A[:4,:4], ref_arr_Data.shape[1])
    # temp_arr_frame2_b = np.ones((ref_arr_Data.shape[0], ref_arr_Data.shape[1])) * temp_arr_frame2_b[:,0].reshape(ref_arr_Data.shape[0], 1)
    # temp_arr_frame2_A = np.tile(temp_arr_frame2_A[:4,:4], ref_arr_Data.shape[1])

    # np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame1_b.txt", temp_arr_frame1_b, fmt='%.5f')
    # np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame1_A.txt", temp_arr_frame1_A, fmt='%.5f')
    # np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame2_b.txt", temp_arr_frame2_b, fmt='%.5f')
    # np.savetxt(TPGMM_dir + demons_nums[d] + "_sample_frame2_A.txt", temp_arr_frame2_A, fmt='%.5f')

    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_b.txt", 'r') as f:
    #     data = f.read(); data = data.replace(' ', ',')
    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_b.txt", 'w') as f:
    #     f.write(data)

    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_A.txt", 'r') as f:
    #     data = f.read(); data = data.replace(' ', ',')
    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame1_A.txt", 'w') as f:
    #     f.write(data)

    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame2_b.txt", 'r') as f:
    #     data = f.read(); data = data.replace(' ', ',')
    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame2_b.txt", 'w') as f:
    #     f.write(data)

    # with open(r"{}".format(TPGMM_dir) + demons_nums[d] + "_sample_frame2_A.txt", 'r') as f:
    #     data = f.read(); data = data.replace(' ', ',')
    # with open(r"{}".format(TPGMM_dir) + demons_nums[d]+ "_sample_frame2_A.txt", 'w') as f:
    #     f.write(data)

    ## Only for debuggin to check the txt file shape
    print("Debugging")
    temp_arr_Data = np.loadtxt(TPGMM_dir + demons_nums[d] + "_sample_Data.txt", delimiter=',')
    print(temp_arr_Data.shape)
    print("\n")

    demons_nums_dtw.append(demons_nums[d])
    
# Resend the dictionary again after filtering the Demons according to DTW (i.e. keeping the most similar ones)
ref_demon["demons_nums"] = demons_nums_dtw
ref_demon["nbDemons"] = len(ref_demon["demons_nums"])
print("Reference Demon:")
print(ref_demon)
with open('/home/erl/Multicobot-UR10/src/tp_gmm/scripts/demons_info2.pkl', 'wb') as fp:
    pickle.dump(ref_demon, fp)    


Reference Demon:
(4, 54)
DTWinng..
(4, 54)
(54,)
245.0
12.97751792319842
0.12016220299257796
Debugging
(4, 54)


DTWinng..
(4, 158)
(54,)
2288.872611464968
127.6374925908485
0.6020636442964552
DTWinng..
(4, 54)
(54,)
373.0
11.791452864913094
0.10918011911956568
Debugging
(4, 54)


DTWinng..
(4, 54)
(54,)
108.0
9.610536051100365
0.08898644491759597
Debugging
(4, 54)


DTWinng..
(4, 54)
(54,)
260.0
14.76219809434631
0.13668701939209546
Debugging
(4, 54)


DTWinng..
(4, 122)
(54,)
1536.3636363636363
56.26889394817393
0.3197096247055337
DTWinng..
(4, 54)
(54,)
231.0
28.603486054830153
0.2648470931002792
Debugging
(4, 54)


DTWinng..
(4, 54)
(54,)
147.0
23.897555045333075
0.2212736578271581
Debugging
(4, 54)


DTWinng..
(4, 54)
(54,)
0.0
0.0
0.0
Debugging
(4, 54)


DTWinng..
(4, 57)
(54,)
950.5
71.2559339174818
0.6419453506079441
DTWinng..
(4, 278)
(54,)
4145.909747292419
161.583981402207
0.4866987391632741
DTWinng..
(4, 134)
(54,)
2082.7142857142853
102.70863981679265
0.5463225522169822
DT

In [30]:
# print(temp_arr_frame1_b.shape)
# print(temp_arr_frame1_A.shape)
# print(temp_arr_frame2_b.shape)
# print(temp_arr_frame2_A.shape)
# print(temp_arr_frame2_A[:4,:4].shape)